In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from enum import Enum
import json
import plotly.graph_objects as go

In [3]:
class Material(Enum):
    WATER = 0
    WATER_GRASS = 1
    STONE = 2
    DRIFTWOOD = 3
    PIER = 4
    GROUND = 5

class FishType(Enum):
    BUFFALO_FISH = 0
    YELLOW_PERCH = 1
    REDEAR_SUNFISH = 2

# 映射表头到枚举值
material_mapping = {
    'Water': Material.WATER,
    'Water Grass': Material.WATER_GRASS,
    'Stone': Material.STONE,
    'Driftwood': Material.DRIFTWOOD,
    'Pier': Material.PIER,
    'Ground': Material.GROUND
}

fish_mapping = {
    '水牛鱼': FishType.BUFFALO_FISH,
    '小冠太阳鱼': FishType.REDEAR_SUNFISH,
    '黄金鲈': FishType.YELLOW_PERCH
}

# 反向映射枚举值到表头
reverse_material_mapping = {v: k for k, v in material_mapping.items()}
reverse_fish_mapping = {v: k for k, v in fish_mapping.items()}

material_color_mapping = {
    Material.WATER.value: 'blue',
    Material.WATER_GRASS.value: 'green',
    Material.STONE.value: 'white',
    Material.DRIFTWOOD.value: 'brown',
    Material.PIER.value: 'black',
    Material.GROUND.value: 'rgba(0,0,0,0)'  # 使用 'rgba(0,0,0,0)' 表示透明
}

fishTypeCount = FishType.__len__()
noFishIndex = fishTypeCount
print(f'钓空/无鱼权重将被插入到第{noFishIndex}个位置')

material_df = pd.read_excel('exampleTables/exampleData.xlsx', sheet_name='material', index_col=0)
environment_df = pd.read_excel('exampleTables/exampleData.xlsx', sheet_name='environment', index_col=0)
pond_df = pd.read_excel('exampleTables/exampleData.xlsx', sheet_name='pond', index_col=0)
bait_df = pd.read_excel('exampleTables/exampleData.xlsx', sheet_name='bait')

# column name consts
N_FAV_TEMPERATURE = "喜好水温"
N_COEF_TEMPERATURE = "水温-权重衰减系数"
N_FAV_OXYGEN = "喜好含氧量"
N_COEF_OXYGEN = "含氧量-权重衰减系数"
N_VISUAL_LOW = "视觉低光阈值"
N_COEF_SMELL = "嗅觉衰减系数"
N_SMELL_MAX = "嗅觉最大距离"
N_PROB_SCORE_BASELINE = "基准权重"

钓空/无鱼权重将被插入到第3个位置


In [4]:
def plot_numpy_3d_colors(data: np.ndarray, colors: dict, sizeScale: float = 10.0):
    fig = go.Figure()
    
    # 修正 np.arange 的用法
    x_range = np.arange(data.shape[0])
    y_range = np.arange(data.shape[1])
    z_range = np.arange(data.shape[2])

    # 使用 np.meshgrid 创建网格点
    x_points, y_points, z_points = np.meshgrid(x_range, y_range, z_range, indexing='ij')
    x_points = x_points.flatten()
    y_points = y_points.flatten()
    z_points = z_points.flatten()
    
    # 映射颜色
    color_points = np.array([colors[data[x, y, z]] for x, y, z in zip(x_points, y_points, z_points)], dtype=str)
    color_points = color_points.flatten()

    # 添加点到图形中
    fig.add_trace(go.Scatter3d(
        x=x_points,
        y=y_points,
        z=z_points,
        mode='markers',
        marker=dict(
            size=5,
            color=color_points,  # 使用 color_points 来映射颜色
            opacity=0.5  # 设置全局透明度
        )
    ))

    # 设置布局
    fig.update_layout(
        scene=dict(
            xaxis=dict(nticks=10, range=[0, data.shape[0]]),
            yaxis=dict(nticks=10, range=[0, data.shape[1]]),
            zaxis=dict(nticks=5, range=[data.shape[2], 0]),
            aspectmode='manual',
            aspectratio=dict(x=data.shape[0]/sizeScale, y=data.shape[1]/sizeScale, z=data.shape[2]/sizeScale)
        ),
        width=800,
        height=800,
        margin=dict(r=20, l=10, b=10, t=10)
    )

    # 显示图形
    fig.show()

def plotViridis(data: np.ndarray, materials:np.ndarray, upperLimit:float ,sizeScale: float = 10.0):
    fig = go.Figure()
    
    # 修正 np.arange 的用法
    x_range = np.arange(data.shape[0])
    y_range = np.arange(data.shape[1])
    z_range = np.arange(data.shape[2])

    # 使用 np.meshgrid 创建网格点
    x_grid, y_grid, z_grid = np.meshgrid(x_range, y_range, z_range, indexing='ij')
    mask = materials != Material.GROUND.value

    x_points = x_grid[mask]
    y_points = y_grid[mask]
    z_points = z_grid[mask]
    color_points = data[mask]/upperLimit
    
    plt.add_trace(go.Scatter3d(
        x=x_points,
        y=y_points,
        z=z_points,
        mode='markers',
        marker=dict(
            size=5,
            color=color_points,  # 使用 color_points 来映射颜色
            colorscale='Viridis',
            opacity=0.5  # 设置全局透明度
        )
    ))

    # 设置布局
    fig.update_layout(
        scene=dict(
            xaxis=dict(nticks=10, range=[0, data.shape[0]]),
            yaxis=dict(nticks=10, range=[0, data.shape[1]]),
            zaxis=dict(nticks=5, range=[data.shape[2], 0]),
            aspectmode='manual',
            aspectratio=dict(x=data.shape[0]/sizeScale, y=data.shape[1]/sizeScale, z=data.shape[2]/sizeScale)
        ),
        width=800,
        height=800,
        margin=dict(r=20, l=10, b=10, t=10)
    )
    plt.show()

def plotGreys(data: np.ndarray, materials:np.ndarray, upperLimit:float ,sizeScale: float = 10.0):
    fig = go.Figure()
    
    # 修正 np.arange 的用法
    x_range = np.arange(data.shape[0])
    y_range = np.arange(data.shape[1])
    z_range = np.arange(data.shape[2])

    # 使用 np.meshgrid 创建网格点
    x_grid, y_grid, z_grid = np.meshgrid(x_range, y_range, z_range, indexing='ij')
    mask = materials != Material.GROUND.value

    x_points = x_grid[mask]
    y_points = y_grid[mask]
    z_points = z_grid[mask]
    color_points = data[mask]/upperLimit
    
    plt.add_trace(go.Scatter3d(
        x=x_points,
        y=y_points,
        z=z_points,
        mode='markers',
        marker=dict(
            size=5,
            color=color_points,  # 使用 color_points 来映射颜色
            colorscale='Greys',
            opacity=0.5  # 设置全局透明度
        )
    ))

    # 设置布局
    fig.update_layout(
        scene=dict(
            xaxis=dict(nticks=10, range=[0, data.shape[0]]),
            yaxis=dict(nticks=10, range=[0, data.shape[1]]),
            zaxis=dict(nticks=5, range=[data.shape[2], 0]),
            aspectmode='manual',
            aspectratio=dict(x=data.shape[0]/sizeScale, y=data.shape[1]/sizeScale, z=data.shape[2]/sizeScale)
        ),
        width=800,
        height=800,
        margin=dict(r=20, l=10, b=10, t=10)
    )
    plt.show()
    

In [5]:
with open('map_OL2.0_v1_159x64.json', 'r') as f:
    map_data = json.load(f)

mapDatas = map_data['mapDatas']

# 探寻最大水深
maxDepth = 0
maxX = 0
maxY = 0

for rows in mapDatas:
    for item in rows:
        maxDepth = max(maxDepth, item['waterDepth'])
        maxX = max(maxX, item['x'])
        maxY = max(maxY, item['y'])

# 定义湖的尺寸
map_length = maxX + 1  # x方向的长度
map_width = maxY + 1  # y方向的宽度
map_depth = maxDepth    # z方向的深度

print(f'湖的尺寸为 {map_length} x {map_width} x {map_depth}')

湖的尺寸为 159 x 64 x 5


In [6]:
material_np = np.full((map_length, map_width, map_depth), Material.GROUND.value, dtype=int)

for rows in mapDatas:
    for item in rows:
        x = item['x']
        y = item['y']
        depth = item['waterDepth']
        # 先初始化成2d编辑器编出来的material
        for i in range(depth):
            material_np[x][y][i] = Material.WATER.value
        # 如果有表面物体，就直通到水底
        if item['surfaceType'] != 0:
            for i in range(0, depth):
                material_np[x][y][i] = item['surfaceType']
        # bottom barriers
        if item['underWaterType'] != 0:
            material_np[x][y][depth-1] = item['underWaterType']

# plot_numpy_3d_colors(material_np, material_color_mapping, 10.0)

# Process fish config DATA

In [7]:
pondListDf = pd.read_excel('configs/xlsx_channel/2001/fish_pond.xlsx', sheet_name='PondList', header=1, skiprows=[2,3])
pondListDf.head()


,id,name,pond_name,pond_img,map_id,fish_stock_id,entry_level,entry_item,entry_fee,open_spot_list,open_spot_list.1,open_spot_list.2,hypolimnion_t,mark
0,301020000,yinchuan_1,yinchuan_1,picture_scene_1,map_base_1,fish_stock_all,1,item_currency_coins,10,1.0,NaN,NaN,120,金币
1,301020001,yinchuan_2,yinchuan_2,picture_scene_2,map_base_2,fish_stock_all,2,item_currency_coins,100,NaN,2.0,3.0,120,金币
2,301020002,yinchuan_3,yinchuan_3,picture_scene_3,map_base_3,fish_stock_null,3,item_currency_coins,100,1.0,2.0,3.0,120,金币


In [8]:
theStockId = pondListDf.loc[pondListDf['name'] == 'yinchuan_1', 'fish_stock_id'].values[0]
print(theStockId)

fish_stock_all


In [9]:
fishReleaseDf = pd.read_excel('configs/xlsx_channel/2001/fish_pond.xlsx', sheet_name='FishRelease', header=1, skiprows=[2,3])
fishReleaseDf.head()


,id,stock_id,fish_id,fish_num,prob_weight_ideal,min_env_coeff,min_adapt_coeff,length_min,length_max,is_reduce
0,1,fish_stock_all,Fish_Largemouth_Bass_Young,988,988,NaN,NaN,10,25,[表达式]否
1,2,fish_stock_all,Fish_Largemouth_Bass_Common,292,292,NaN,NaN,25,40,[表达式]否
2,3,fish_stock_all,Fish_Largemouth_Bass_Trophy,48,48,NaN,NaN,40,50,[表达式]否
3,4,fish_stock_all,Fish_Largemouth_Bass_Unique,9,9,NaN,NaN,50,74,[表达式]否
4,5,fish_stock_all,Fish_Spotted_Bass_Young,1179,1179,NaN,NaN,10,17,[表达式]否


In [10]:
fishReleaseDf = fishReleaseDf[fishReleaseDf['stock_id'] == theStockId]
print(fishReleaseDf.count())

fishReleaseDf.drop(columns=['stock_id', 'id', 'fish_num', 'is_reduce'], inplace=True)
# make NaN to 0
fishReleaseDf.fillna(0, inplace=True)

print(fishReleaseDf.head())




id                   58
stock_id             58
fish_id              58
fish_num             58
prob_weight_ideal    58
min_env_coeff         0
min_adapt_coeff       0
length_min           58
length_max           58
is_reduce            58
dtype: int64
                       fish_id  prob_weight_ideal  min_env_coeff  \
0   Fish_Largemouth_Bass_Young                988            0.0   
1  Fish_Largemouth_Bass_Common                292            0.0   
2  Fish_Largemouth_Bass_Trophy                 48            0.0   
3  Fish_Largemouth_Bass_Unique                  9            0.0   
4      Fish_Spotted_Bass_Young               1179            0.0   

   min_adapt_coeff  length_min  length_max  
0              0.0          10          25  
1              0.0          25          40  
2              0.0          40          50  
3              0.0          50          74  
4              0.0          10          17  


In [11]:
fishNames = fishReleaseDf['fish_id'].unique()
print(fishNames.shape) 

print(fishNames[0])


(58,)
Fish_Largemouth_Bass_Young


In [12]:
fishEnvAffinityDf = pd.read_excel('configs/xlsx_channel/2001/fish_env_affinity.xlsx', sheet_name='FishEnvAffinity', header=1, skiprows=[2,3])
fishEnvAffinityDf.index = fishEnvAffinityDf['name']
fishEnvAffinityDf.drop(columns=['name'], inplace=True)
fishEnvAffinityDf.head()

,id,fish_quality,struct_id,temp_id,layer_id,bait_coeff,bait_type_coeff,mark
name,,,,,,,,
Fish_Largemouth_Bass_Young,201020001,Largemouth_Bass_Young,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,暖水植被区表中层掠食鱼(Centrarchidae)
Fish_Largemouth_Bass_Common,201020002,Largemouth_Bass_Common,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,同上，成熟期
Fish_Largemouth_Bass_Trophy,201020003,Largemouth_Bass_Trophy,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,奖杯级别，同科同生态
Fish_Largemouth_Bass_Unique,201020004,Largemouth_Bass_Unique,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,稀有个体，生态不变
Fish_Spotted_Bass_Young,201020005,Spotted_Bass_Young,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,与大口黑鲈类似(Centrarchidae)


In [13]:
# choose rows where index (name) equals fishNames(strictly in fishNames order)
selectedFishEnvAffinityDf = fishEnvAffinityDf.loc[fishNames]
selectedFishEnvAffinityDf.head()



,id,fish_quality,struct_id,temp_id,layer_id,bait_coeff,bait_type_coeff,mark
name,,,,,,,,
Fish_Largemouth_Bass_Young,201020001,Largemouth_Bass_Young,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,暖水植被区表中层掠食鱼(Centrarchidae)
Fish_Largemouth_Bass_Common,201020002,Largemouth_Bass_Common,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,同上，成熟期
Fish_Largemouth_Bass_Trophy,201020003,Largemouth_Bass_Trophy,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,奖杯级别，同科同生态
Fish_Largemouth_Bass_Unique,201020004,Largemouth_Bass_Unique,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,稀有个体，生态不变
Fish_Spotted_Bass_Young,201020005,Spotted_Bass_Young,structure_bass,temp_22_3.5,water_layer_bass,89757,1001,与大口黑鲈类似(Centrarchidae)


In [14]:
tempIds = selectedFishEnvAffinityDf['temp_id'].values
print(tempIds.shape)

(58,)


In [15]:
tempDf = pd.read_excel('configs/xlsx_channel/2001/fish_env_affinity.xlsx', sheet_name='TempAffinity', header=1, skiprows=[2,3])

tempDf = tempDf.set_index('name')

tempDf = tempDf.loc[tempIds]
temperature_fav_np = tempDf['temperature_fav'].values
temp_affected_ratio_np = tempDf['temp_affected_ratio'].values


In [16]:
structDf = pd.read_excel('configs/xlsx_channel/2001/fish_env_affinity.xlsx', sheet_name='StructAffinity', header=1, skiprows=[2,3])
structDf.set_index('name', inplace=True)
structDf.head()

,id,struct_type,coeff,struct_type.1,coeff.1,struct_type.2,coeff.2,struct_type.3,coeff.3,struct_type.4,coeff.4,mark
name,,,,,,,,,,,,
structure_all_1,1,[水下结构体]开放水域,1.0,[水下结构体]水草,1.0,[水下结构体]石头,1.0,[水下结构体]沉木,1.0,[水下结构体]桥墩,1.0,通用占位
structure_bass,2,[水下结构体]开放水域,0.6,[水下结构体]水草,1.0,[水下结构体]石头,0.5,[水下结构体]沉木,0.8,[水下结构体]桥墩,0.5,适合大口黑鲈
structure_walleye,3,[水下结构体]开放水域,0.7,[水下结构体]水草,0.4,[水下结构体]石头,1.0,[水下结构体]沉木,0.4,[水下结构体]桥墩,0.5,适合Walleye
structure_catfish,4,[水下结构体]开放水域,0.7,[水下结构体]水草,0.5,[水下结构体]石头,0.6,[水下结构体]沉木,0.6,[水下结构体]桥墩,0.5,Channel Catfish等
structure_buffalo,5,[水下结构体]开放水域,0.8,[水下结构体]水草,0.5,[水下结构体]石头,0.5,[水下结构体]沉木,0.5,[水下结构体]桥墩,0.4,Buffalo鱼


In [17]:

# 创建一个空列表来存储重组后的数据
restructured_data = []

# 遍历每一行数据
for index, row in structDf.iterrows():
    name = index
    # 处理每对struct_type和coeff
    for i in range(1, 6):  # 5组struct_type和coeff
        struct_type = row[f'struct_type.{i}'] if f'struct_type.{i}' in row else row['struct_type']
        coeff = row[f'coeff.{i}'] if f'coeff.{i}' in row else row['coeff']
        
        # 从struct_type中提取纯结构体名称（去掉"[水下结构体]"前缀）
        struct_name = struct_type.replace('[水下结构体]', '')
        
        restructured_data.append({
            'name': name,
            'structure': struct_type,
            'coefficient': coeff
        })

# 创建新的DataFrame
new_df = pd.DataFrame(restructured_data)

# 透视表转换
pivot_table = new_df.pivot(
    index='name',
    columns='structure',
    values='coefficient'
)

# 打印结果
print(pivot_table)

structure              [水下结构体]开放水域  [水下结构体]桥墩  [水下结构体]水草  [水下结构体]沉木  [水下结构体]石头
name                                                                          
structure_all_1                1.0        1.0        1.0        1.0        1.0
structure_bass                 0.6        0.5        1.0        0.8        0.5
structure_bowfin               0.5        0.5        0.8        0.7        0.5
structure_buffalo              0.8        0.4        0.5        0.5        0.5
structure_catfish              0.7        0.5        0.5        0.6        0.6
structure_drum                 0.7        0.4        0.5        0.5        0.6
structure_muskellunge          0.5        0.5        0.8        0.6        0.5
structure_tench                0.5        0.4        0.8        0.6        0.4
structure_trout                0.6        0.5        0.5        0.5        0.8
structure_walleye              0.7        0.5        0.4        0.4        1.0


In [18]:
structAffIds = fishEnvAffinityDf['struct_id'].values
fishToStructAffDf = pivot_table.loc[structAffIds]
fishToStructAffDf

structure,[水下结构体]开放水域,[水下结构体]桥墩,[水下结构体]水草,[水下结构体]沉木,[水下结构体]石头
name,,,,,
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_bass,0.6,0.5,1.0,0.8,0.5
structure_walleye,0.7,0.5,0.4,0.4,1.0


In [23]:
# 读取Excel文件
df = pd.read_excel('configs/xlsx_channel/2001/enum.xlsx', header=None)

# 初始化一个字典来存储结果
struct_dict = {}
is_struct_section = False

# 遍历DataFrame的行
for _, row in df.iterrows():
    # 检查是否到了水下结构体部分
    if row[0] == '[水下结构体类型]':
        is_struct_section = True
        continue
    
    # 如果是空行或遇到下一个分类，则停止
    if pd.isna(row[0]) or (row[0].startswith('[') and not row[0].startswith('[水下结构体]')):
        is_struct_section = False
        continue
        
    # 如果在水下结构体部分，且行不为空，则提取数据
    if is_struct_section and not pd.isna(row[0]):
        name = row[0]
        value = int(row[1])  # 获取对应的数值
        struct_dict[name] = value

# 打印结果
print(struct_dict)

{'[水下结构体]开放水域': 0, '[水下结构体]水草': 1, '[水下结构体]石头': 2, '[水下结构体]沉木': 3, '[水下结构体]桥墩': 4}


In [30]:
# 检查fishToStructAffDf的列名是否按照struct_dict的顺序排列
columns_list = fishToStructAffDf.columns.tolist()
print(columns_list)
print([struct_dict[col] for col in columns_list])

['[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩']
[0, 1, 2, 3, 4]


In [29]:
# 重新排序列名
new_columns_list = sorted(columns_list, key=lambda x: struct_dict[x])
fishToStructAffDf = fishToStructAffDf[new_columns_list]
fishToStructAffDf

structure,[水下结构体]开放水域,[水下结构体]水草,[水下结构体]石头,[水下结构体]沉木,[水下结构体]桥墩
name,,,,,
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_bass,0.6,1.0,0.5,0.8,0.5
structure_walleye,0.7,0.4,1.0,0.4,0.5


In [35]:
# 用同样的方式处理wterLayerAffinity
waterLayerDf = pd.read_excel('configs/xlsx_channel/2001/fish_env_affinity.xlsx', sheet_name='WaterLayerAffinity', header=1, skiprows=[2,3])
waterLayerDf.set_index('name', inplace=True)

# make pivot table
restructured_data = []

for index, row in waterLayerDf.iterrows():
    name = index
    for i in range(1, 4):
        layer_type = row[f'layer_type.{i}'] if f'layer_type.{i}' in row else row['layer_type']
        coeff = row[f'coeff.{i}'] if f'coeff.{i}' in row else row['coeff']
        
        layer_name = layer_type.replace('[水层]', '')
        
        restructured_data.append({
            'name': name,
            'layer': layer_type,
            'coefficient': coeff
        })

new_df = pd.DataFrame(restructured_data)
pivot_table_water_layer = new_df.pivot(
    index='name',
    columns='layer',
    values='coefficient'
)

waterLayerIds = fishEnvAffinityDf['layer_id'].values
fishToWaterLayerAffDf = pivot_table_water_layer.loc[waterLayerIds]
fishToWaterLayerAffDf

layer,[地图水层]中层,[地图水层]底层,[地图水层]表层
name,,,
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_bass,0.7,0.3,0.9
water_layer_walleye,0.5,0.9,0.2


In [37]:
enumsDf = pd.read_excel('configs/xlsx_channel/2001/enum.xlsx', header=None)

# 初始化一个字典来存储结果
water_layer_dict = {}

is_water_layer_section = False

for _, row in enumsDf.iterrows():
    if row[0] == '[地图水层类型]':
        is_water_layer_section = True
        continue
    
    if pd.isna(row[0]) or (row[0].startswith('[') and not row[0].startswith('[地图水层]')):
        is_water_layer_section = False
        continue
        
    if is_water_layer_section and not pd.isna(row[0]):
        name = row[0]
        value = int(row[1])
        water_layer_dict[name] = value

# make sure the order of columns is correct
columns_list = fishToWaterLayerAffDf.columns.tolist()
print(columns_list)

new_columns_list = sorted(columns_list, key=lambda x: water_layer_dict[x])
fishToWaterLayerAffDf = fishToWaterLayerAffDf[new_columns_list]
fishToWaterLayerAffDf

['[地图水层]中层', '[地图水层]底层', '[地图水层]表层']


layer,[地图水层]表层,[地图水层]中层,[地图水层]底层
name,,,
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_bass,0.9,0.7,0.3
water_layer_walleye,0.2,0.5,0.9


In [ ]:
# 设置temp_pond_np的shape与material_np相同
temp_pond_np = np.zeros(material_np.shape, dtype=float)
waterlayer_pond_np = np.zeros(material_np.shape, dtype=np.int64)
# waterlayer_pond_np = waterlayer_pond_np.astype(np.int32)

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations